In [ ]:
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import optuna

In [ ]:
df_train = pd.read_csv('../databases/training.csv')
df_valid = pd.read_csv('../databases/validation.csv')

In [ ]:
df_train = df_train.drop(['IND_BOM_1_2'], axis=1)
df_valid = df_valid.drop(['IND_BOM_1_2'], axis=1)

In [ ]:
X_train, y_train = df_train.drop(['IND_BOM_1_1'], axis=1), df_train['IND_BOM_1_1']
X_val, y_val = df_valid.drop(['IND_BOM_1_1'], axis=1), df_valid['IND_BOM_1_1']

In [ ]:
params = {
    'hidden_layer_sizes': [32, 64, 128],
    'alpha': [0.001, 0.01, 0.1],
    'max_iter': [2000, 5000, 10000],
    'batch_size': [32, 64, 128],
    'activation': ['tanh', 'relu', 'sigmoid']
}

def objective(trial):
    model = keras.Sequential([
        keras.layers.Dense(
            units=trial.suggest_categorical(
                'hidden_layer_sizes',
                params['hidden_layer_sizes']
            ),
            activation=trial.suggest_categorical(
                'activation',
                params['activation']
            )
        ),
        keras.layers.Dense(
            units=trial.suggest_categorical(
                'hidden_layer_sizes',
                params['hidden_layer_sizes']
            ),
            activation=trial.suggest_categorical(
                'activation',
                params['activation']
            )
        ),
        keras.layers.Dense(units=1, activation='sigmoid')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(
            trial.suggest_float(
                'alpha',
                low=params['alpha'][0],
                high=params['alpha'][2]
            )
        ),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )

    model.fit(
        X_train,
        y_train,
        batch_size=trial.suggest_categorical(
            'batch_size',
            params['batch_size']
        ),
        epochs=trial.suggest_categorical(
            'max_iter',
            params['max_iter']
        ),
        validation_data=(X_val, y_val)
    )

    loss, accuracy = model.evaluate(X_val, y_val)

    return 1 - accuracy

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
print('Best hyperparameters: ', study.best_params)
print('Best objective value: ', study.best_value)